<a href="https://colab.research.google.com/github/ksaisudha24/idScanner/blob/master/idScanner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**INSTALLING PACKAGES**

In [ ]:
!pip install anvil-uplink

     |████████████████████████████████| 64 kB 1.8 MB/s 
     |████████████████████████████████| 51 kB 120 kB/s 
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45229 sha256=ed14d42005077012252e68cf3c7fc0c2daeece1d577be8a682449c99dd46bfa6
  Stored in directory: /root/.cache/pip/wheels/29/ea/7d/3410aa0aa0e4402ead9a7a97ab2214804887e0f5c2b76f0c96
Successfully built ws4py


In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 37 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 3s (1,568 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl

**IMPORTING PACKAGES**

In [ ]:
import cv2
import pytesseract
import numpy as np
from PIL import Image
import anvil.server
anvil.server.connect("RWFIVIQ4QFHCNNSUGNW5A75Z-SVRWJCQTMVBAYHTV")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment" as SERVER


**CAMERA CAPTURE**

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

**ROLL NUMBER AUTHENTICATION**

In [ ]:
#read img
def authenticate(file):
  imgUMat = cv2.imread(file)
  gray = cv2.cvtColor(imgUMat, cv2.COLOR_BGR2GRAY)

  ## (2) Threshold
  th, threshed = cv2.threshold(gray, 127, 255, cv2.THRESH_TRUNC)

  ## (3) Detect
  result = pytesseract.image_to_string(threshed)
  lst = result.split()

  if "Roll" in lst:
    ind = lst.index("Roll")
  elif "Roli" in lst:
    ind = lst.index("Roli")
  else:
    return "Not Authenticated"

  roll_numbers = ["199X1A0575", "199X1A0123", "199X1AQ575"]
  if len(lst[ind + 1]) == 3:
    req = lst[ind + 2]
  else:
    req = lst[ind + 1][3:]
  print(req)

  if req in roll_numbers:
    return "Authenticated"
  return "Not Authenticated"



In [ ]:
@anvil.server.callable
def auth():
  # file = take_photo()
  # return authenticate(file)
  return authenticate("college-id.jpeg")

In [ ]:
anvil.server.wait_forever()

199X1A0575
